# Prime fields
Let's use our module to build Frobenius fields, aka prime fields $ \mathbf{F}_p $.

First, we need to load the functor that makes a field based on a characteristic `carac`, which must be a prime.

In [1]:
#use "./src/frobenius.ml"

	Camlp4 Parsing version 4.02.3

module type FrobT =
  sig
    type elt
    val zero : elt
    val un : elt
    val alpha : elt
    val carac : int
    val dim : int
    val cardinal : int
    val int_of_elt : elt -> int
    val elt_of_int : int -> elt
    val list_of_elt : elt -> int list
    val elt_of_list_int : int list -> elt
    val print : elt -> unit
    val print_table : unit -> unit
    val add : elt -> elt -> elt
    val sub : elt -> elt -> elt
    val opp : elt -> elt
    val mult : elt -> elt -> elt
    val inv : elt -> elt
    val div : elt -> elt -> elt
    val pow : elt -> int -> elt
    val random : unit -> elt
  end
module Frobenius : functor (Taille : sig val carac : int end) -> FrobT


Now, let's build $ \mathbf{F}_3  = \mathbf{Z}/(3\mathbf{Z})$ :

In [2]:
module T3 = struct
    let carac = 3
end

module F3 = Frobenius(T3)

module T3 : sig val carac : int end


module F3 :
  sig
    type elt = Frobenius(T3).elt
    val zero : elt
    val un : elt
    val alpha : elt
    val carac : int
    val dim : int
    val cardinal : int
    val int_of_elt : elt -> int
    val elt_of_int : int -> elt
    val list_of_elt : elt -> int list
    val elt_of_list_int : int list -> elt
    val print : elt -> unit
    val print_table : unit -> unit
    val add : elt -> elt -> elt
    val sub : elt -> elt -> elt
    val opp : elt -> elt
    val mult : elt -> elt -> elt
    val inv : elt -> elt
    val div : elt -> elt -> elt
    val pow : elt -> int -> elt
    val random : unit -> elt
  end


In [3]:
F3.(print alpha)

- : unit = ()


Unfortunately, we cannot print in here directly. We'll just need to flush standard output, which can be done easily  done `print_newline()` or `print_endline()` cf https://caml.inria.fr/pub/docs/manual-ocaml/libref/Pervasives.html :

In [4]:
print_newline ()

2%3


- : unit = ()


We can now see that the 2 is a (the) primitive element in this field. 
If we need more flexible representation, we can just use other primitives :

In [5]:
F3.(int_of_elt alpha) |> print_int ; print_newline ()

2


- : unit = ()


Let's extend this field to make $ \mathbf{F}_9 $. To do this, we're actually going to build $ \mathbf{F}_3[X]/(f(x))$ where $f(x)$ is an irreducible polynomial of degree 2.
First, we need to make polynoms out of these. Then, the module Cantorzass factorizes a cyclotomic polynomial and exten

In [6]:
#mod_use "./src/polynome.ml"
#mod_use "./src/cantorzass.ml"
#mod_use "./src/cyclo.ml"
#use "./src/extensions.ml"

module Polynome :
  sig
    module type CorpsType =
      sig
        type elt
        val carac : int
        val cardinal : int
        val dim : int
        val zero : elt
        val un : elt
        val random : unit -> elt
        val print : elt -> unit
        val opp : elt -> elt
        val inv : elt -> elt
        val add : elt -> elt -> elt
        val mult : elt -> elt -> elt
        val sub : elt -> elt -> elt
        val div : elt -> elt -> elt
        val pow : elt -> int -> elt
      end
    module type PolyFunctorType =
      functor (Corps : CorpsType) ->
        sig
          type elt = Corps.elt
          type poly
          val carac : int
          val cardinal : int
          val poly_of_list : elt list -> poly
          val list_of_poly : poly -> elt list
          val nul : poly
          val un : poly
          val coef_dom : poly -> elt
          val map : (elt -> elt) -> poly -> poly
          val evaluer : poly -> elt -> elt
          val normalise : poly 

In [7]:
module T6 = struct
    let carac = 3
    let dim = 2
end

module F9 = ExtensionOpt(F3)(T6)

module T6 : sig val carac : int val dim : int end


module F9 :
  sig
    type elt = ExtensionOpt(F3)(T6).elt
    exception ExtraireImpossible
    val carac : int
    val dim : int
    val cardinal : int
    val zero : elt
    val elt_of_int : int -> elt
    val int_of_elt : elt -> int
    val list_of_elt : elt -> int list
    val elt_of_list_int : int list -> elt
    val extraire : elt -> F3.elt
    val incorporer : F3.elt -> elt
    val un : elt
    val alpha : elt
    val add : elt -> elt -> elt
    val sub : elt -> elt -> elt
    val opp : elt -> elt
    val mult : elt -> elt -> elt
    val inv : elt -> elt
    val div : elt -> elt -> elt
    val pow : elt -> int -> elt
    val print : elt -> unit
    val print_table : unit -> unit
    val random : unit -> elt
  end


In [12]:
let _ = 
    let open F9 in
    let rec enum x = match x with
        | a when a=un -> print un ; print_newline() ;
        | a -> print a ; print_newline() ; enum (mult a alpha)
    in enum alpha
;
print_newline() ; F9.print_table ()

- : unit = ()


a^1
a^2
a^3
a^4
a^5
a^6
a^7
a^0

1%3X^0

0%3X^0 + 1%3X^1

1%3X^0 + 2%3X^1

2%3X^0 + 2%3X^1

2%3X^0

0%3X^0 + 2%3X^1

2%3X^0 + 1%3X^1

1%3X^0 + 1%3X^1

